In [10]:
#openai==v1.3最新版2023update対応、簡易AI議事録

import gradio as gr
from pydub import AudioSegment
import os
from openai import OpenAI
import time

# OpenAIのクライアントを初期化
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY3']
)

#要約prompt
integrated_template = """
次の文章は議事録の文字起こしです。以下の手順に従って下さい。

# 手順1: 議事録の要点をまとめる
## 制約条件
* 発言の内容から言いたいことを抽出し、議事録の内容に基づいて、理由と共にまとめる。その度に改行すること。
* 関連した論点などが重複している場合は総合して一つの要点とすること
* それぞれの要点は300字程度でまとめること
* 明確で意味がわかりやすい文章にすること

## 議事録の要点
文章：{text}
要点：

# 手順2: 議事録の要点に基づいてサマリーをMarkdown形式で作成してください。
## サマリーのフォーマットは以下のように
* 会議の目的
* 決定事項
* 今後の課題
* Todo

## 最後に全体的なサマリーを簡潔に書く
"""


# 'tmp' フォルダの中身をクリアする関数
def clear_tmp_folder():
    tmp_folder = "tmp"
    os.makedirs(tmp_folder, exist_ok=True)
    for file_name in os.listdir(tmp_folder):
        file_path = os.path.join(tmp_folder, file_name)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

# 音声ファイルを分割し、各チャンクを文字起こしする関数
def get_duration(inputs):
    audio = AudioSegment.from_file(inputs, format="m4a")
    length = len(audio)
    chunk_length = 1000000
    overlap_length = 10000
    chunks = []

    for i in range(0, length, chunk_length - overlap_length):
        chunk = audio[i:i+chunk_length]
        chunks.append(chunk)

    for i, chunk in enumerate(chunks, start=1):
        output_file = f"tmp/split{i}.mp3"
        chunk.export(output_file, format="mp3")
        
    transcripts = []
    for i in range(1, len(chunks) + 1):
        split_audio_file = f"tmp/split{i}.mp3"
        with open(split_audio_file, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                file=audio_file,
                model="whisper-1",
                response_format="text",
                language="ja"
            )
            transcripts.append(transcript)

        with open(f"tmp/split{i}.txt", "w", encoding="utf-8") as file:
            file.write(transcript)

    with open("tmp/transcript.txt", "w", encoding="utf-8") as output_file:
        for i in range(1, len(chunks) + 1):
            with open(f"tmp/split{i}.txt", "r", encoding="utf-8") as input_file:
                output_file.write(input_file.read())
                output_file.write("\n")

    print("すべてのテキストファイルが transcript.txt に結合されました。")
    time.sleep(2)


# # 最初にしゃべり言葉を適切な文章に修正、ここのルーチンはAPIのトークン数が足りません。また、langchainが最新のopenaiモジュールに対応しないので分割ルーチンも実装不可能となっています。
#     with open("tmp/transcript.txt") as f:
#         transcript = f.read()
    
#     fix_template = fix_talk_template.format(text=transcript)
    
#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo-1106",  # モデル名の確認
#         messages=[
#             {"role": "system", "content": fix_template},
#             {"role": "user", "content": transcript}
#         ],
#         max_tokens=16385,  # 値の修正
#         temperature=0.7,
#     )  
#     fix_transcript = response.choices[0].message.content.strip()
# # 修正されたテキストをファイルに保存
#     with open("tmp/fix_transcript.txt", mode="w", encoding="utf-8") as f:
#         f.write(fix_transcript)

    
#会議のサマリーを作成  
    with open("tmp/transcript.txt") as f:
        transcript = f.read()
    
    formatted_template = integrated_template.format(text=transcript)
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",#このモデルは入力トークン120k、出力4096です。それ以上の議事録はトークンオーバーエラー
        messages=[
            {"role": "system", "content": formatted_template},
            {"role": "user", "content": transcript}
        ],
        max_tokens=4096,  # この値は必要に応じて調整
        temperature=0.7,
    )

    summary = response.choices[0].message.content.strip() 
    
    return summary,transcript


# Clear 'tmp' folder before launching the interface
clear_tmp_folder()

iface = gr.Interface(
    fn=get_duration,
    inputs=gr.Audio(type="filepath", label="会議ファイルをアップロード"),
    outputs=["text", "text"],  # 戻り値が2つあるため、"outputs"パラメータも2つにする
    #theme='freddyaboulton/dracula_revamped',
    #theme='HaleyCH/HaleyCH_Theme',
    theme='freddyaboulton/dracula_revamped',
    title="AI議事録書き起こし、要点書き出し、サマリー生成",
    description="Upload an audio file to Generate."
)

iface.launch(inbrowser=True)


Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


すべてのテキストファイルが transcript.txt に結合されました。
